In [8]:
#   Ejercicio 3

from pymongo import MongoClient
import json

client = MongoClient("mongodb://mongo:27017")

def save_to_txt(filename, data):
    with open(filename, "w", encoding="utf-8") as f:
        for doc in data:
            f.write(json.dumps(doc, default=str, indent=2) + "\n")

db = client["mi_base"]
proveedores = db["proveedores"]

query3 = proveedores.aggregate([
    {"$unwind": "$telefonos"},
    {"$project": {
        "_id": 0,
        "razon_social": 1,
        "CUIT": 1,
        "telefono": "$telefonos"
    }}
])
save_to_txt("query3.txt", list(query3))

In [2]:
#   Ejercicio 6

from pymongo import MongoClient

client = MongoClient("mongodb://mongo:27017")

In [4]:
#   Ejercicio 9

from pymongo import MongoClient

client = MongoClient("mongodb://mongo:27017")

In [ ]:
#   Ejercicio 12

from pymongo import MongoClient

client = MongoClient("mongodb://mongo:27017")

In [6]:
#   Ejercicio 15

from pymongo import MongoClient

client = MongoClient("mongodb://mongo:27017")